## PROBLEM 5: DBSCAN on toy-neighborhood data

You are to cluster and visualize a small dataset using DBSCAN with epsilon = 7.5 and MinPts = 3. The dataset is provided in the file `dbscan.csv`, which contains the following columns for each data point:

- `cluster`: Initially empty, provided for convenience.
- `pt`: Unique ID for each data point.
- `x`: Point's x-coordinate.
- `y`: Point's y-coordinate.
- `num neighbors`: Number of neighbors, according to the coordinates above.
- `neighbors`: IDs of all neighbors within the epsilon radius.

As you can see, a tedious O(n^2) portion of the work has already been done for you. Your job is to execute the DBSCAN algorithm point-by-point, logging your work.


In [149]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import plotly.express as px

In [150]:
df = pd.read_csv('dbscan.csv')
X = df[['x', 'y']].to_numpy()

In [152]:
import numpy as np
import pandas as pd


class DBSCAN:
    def __init__(self, epsilon, minPts):
        self.epsilon = epsilon
        self.minPts = minPts
        self.labels = None
        self.data_frame = None

    def fit(self, X, distance_matrix):
        num_points = distance_matrix.shape[0]
        visited = np.zeros(num_points, dtype=bool)
        self.labels = np.zeros(num_points, dtype=int)
        cluster_label = 0
        output = []
        point_coordinates = np.array([[x, y] for x, y in zip(X[:, 0], X[:, 1])])

        for i in range(num_points):
            if visited[i]:
                continue

            visited[i] = True
            neighbors = self.region_query(i, distance_matrix)

            if len(neighbors) < self.minPts:
                self.labels[i] = -1
            else:
                cluster_label += 1
                self.expand_cluster(i, neighbors, cluster_label, visited, distance_matrix)

        for i in range(num_points):
            neighbor_indices = [str(x) for x in self.region_query(i, distance_matrix)]
            output.append([i, point_coordinates[i, 0], point_coordinates[i, 1], len(neighbor_indices), ','.join(neighbor_indices)])

        self.data_frame = pd.DataFrame(output, columns=["pt", "x", "y", "num_neighbors", "neighbors"])

    def expand_cluster(self, point_index, neighbors, cluster_label, visited, distance_matrix):
        self.labels[point_index] = cluster_label

        while len(neighbors) > 0:
            current_point = neighbors[0]
            neighbors = neighbors[1:]

            if not visited[current_point]:
                visited[current_point] = True
                current_neighbors = self.region_query(current_point, distance_matrix)

                if len(current_neighbors) >= self.minPts:
                    neighbors.extend(current_neighbors)

            if self.labels[current_point] == 0:
                self.labels[current_point] = cluster_label

    def region_query(self, point_index, distance_matrix):
        neighbors = []
        for i, distance in enumerate(distance_matrix[point_index]):
            if distance <= self.epsilon:
                neighbors.append(i)
        return neighbors

In [153]:
from scipy.spatial.distance import cdist

# Calculate the distance matrix using the desired distance metric 
distance_matrix = cdist(X, X, metric='euclidean')


In [154]:
distance_matrix.shape

(80, 80)

In [156]:
# initializing the the model
dbscan = DBSCAN(epsilon=7.5, minPts=3)
dbscan.fit(X, distance_matrix)
print("Labels:", dbscan.labels)
print("Data Frame:")
print(dbscan.data_frame)

Labels: [-1  1 -1 -1  1  2  2 -1  2  3  2  2  1 -1  2 -1  2  2 -1  2  2  2  2 -1
 -1  2  2 -1  1  2  2  2  2  3  2 -1 -1  2  2  2  1 -1  2 -1 -1  2  2  2
  2  2  2 -1  2  2  2 -1  1 -1 -1 -1  2 -1 -1  2  2 -1  1  2  2  2  2  2
  2 -1  2  1  2 -1  3 -1]
Data Frame:
    pt          x          y  num_neighbors neighbors
0    0  51.418089  13.593610              2      0,27
1    1  39.132318  -4.419204              3   1,40,75
2    2  47.807515 -25.822561              1         2
3    3  27.699703  53.434193              1         3
4    4  39.860995   5.676871              3   4,56,75
..  ..        ...        ...            ...       ...
75  75  39.659047   0.230178              3    1,4,75
76  76  26.366491   8.798826              3  21,49,76
77  77 -36.184060  44.292045              2     55,77
78  78  44.012085  37.729478              3   9,33,78
79  79  -1.393234 -55.293943              1        79

[80 rows x 5 columns]


In [157]:
results = dbscan.data_frame
results['labels'] = dbscan.labels

fig = px.scatter(results, x="x", y="y", color="labels")
fig.show()